# Berechnung von CO2-Emissionen anhand eines Testdatensatzes

Innerhalb dieses Notebooks wird ein Feature Layer geladen, welches repräsentativ für Surveydaten steht, einige Berechnungen durchgeführt und die Ergebnisse wieder abgespeichert.


#### Run this cell to connect to your GIS and get started:

In [1]:
import warnings
warnings.filterwarnings("ignore")

from arcgis.gis import GIS
gis = GIS("home")

#### Funktion, um Daten zu laden und zu speichern

In [2]:
# import necessary libraries
import numpy as np
import pandas as pd
from arcgis.gis import GIS
import arcgis.features


## Funktion um Feature Layer als Dataframe zu laden bzw. Dataframes als Feature Layer abzuspeichern.
### WICHTIG: Das Feature Layer, welches mit Daten befüllt werden soll, muss bereits existieren

def searchoverwrite(title, data='null', function='overwrite', i_type='Feature Layer'):
    '''
    - title: Titel des benötigten Layers (Name aus ArcGIS Online)
    - data: Name des DataFrames welches den Layer überschreiben soll (benötigt, wenn function='overwrite')
    - function:
        - 'overwrite': default, gesuchter Layer wird überschrieben
        - 'to_dataframe': gesuchter Layer wird in pandas DataFrame umgewandelt
    - i_type: item type; in der Regel Feature Layer (default)
    '''

    search = gis.content.search(query='title:'+title, item_type=i_type)
    
    # Sucht nach dem exakten Titel-Match
    exact_match = None
    for item in search:
        if item.title == title:
            exact_match = item
            break

    # Wenn es keinen exakten Treffer gibt, Abbruch
    if exact_match is None:
        print(f"Kein Layer mit dem exakten Titel {title} gefunden.")
        return

    try:
        layer = exact_match.layers[0]
    except:
        layer = exact_match.tables[0]
    
    if function == 'to_dataframe':
        df = pd.DataFrame.spatial.from_layer(layer)
        return df
    else:
        layer.manager.truncate()
        fset = arcgis.features.FeatureSet.from_dataframe(data)
        layer.edit_features(adds=fset)

### Testdaten laden 
"5dc1e6c2946f48918d1f1cb8d3ad920d" ist die ID des Testdatensatzes. Dies muss ersetzt werden, sobald mit eigenen Daten gearbeitet wird.
Bei unseren testdaten handelt es sich um eine einfache Tabelle, die wir ohne geographische Informationen in ArcGIS online hochgeladen haben. 

mit `tables[0]` liest man Tabellen ein, mit `layers[0]` werden Feature Layer mit geographischen Informationen eingelesen. Bei Surveydaten gibt es immer eine geographische Information (wenn keine erfragt werden ist es [0,0]), deswegen würde hier tables durch layers ersetzt werden

In [3]:
item = gis.content.get("5dc1e6c2946f48918d1f1cb8d3ad920d") # ID des Datensatzes

layer = item.tables[0]
df = pd.DataFrame.spatial.from_layer(layer)
df

,Ich_bin,Wegstrecke,Tage_Fuß,Tage_Rad,Tage_OPNV,Tage_Auto,Autoart,Heft,College,Zeichen,Druck,Papierart,FID
0,Lehrkraft,6.2,0.0,5.0,0.0,0.0,<NA>,0,0,4,2500,Recyclingpapier,1
1,Lehrkraft,5.0,0.0,5.0,0.0,0.0,<NA>,0,10,0,2500,Recyclingpapier,2
2,Schülerin oder Schüler,3.0,0.0,0.0,5.0,0.0,<NA>,0,0,0,20,Recyclingpapier,3
3,Schülerin oder Schüler,1.7,0.0,5.0,0.0,0.0,<NA>,6,2,1,0,"teils, teils",4
4,Schülerin oder Schüler,2.6,2.5,0.0,2.5,0.0,Diesel,4,3,1,24,"teils, teils",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Schülerin oder Schüler,3.0,0.0,4.0,1.0,0.0,E-Auto,2,1,2,0,Recyclingpapier,174
174,Schülerin oder Schüler,1.3,2.0,3.0,0.0,0.0,<NA>,0,4,1,0,"teils, teils",175
175,Schülerin oder Schüler,0.5,0.0,5.0,0.0,0.0,<NA>,3,3,0,20,Recyclingpapier,176
176,Schülerin oder Schüler,5.0,0.0,0.0,5.0,0.0,<NA>,1,0,0,0,"teils, teils",177


### Berechnung Emissionen durch Papierverbrauch

In [4]:
# Heft = 16 Blatt
# Collageblock = 80 Blatt
# Zeichenblock = A3 20 Blatt = A4 40 Blatt
# Druckerpapier = Anzahl steht da
recycling = 0.0041  #1 Blatt Papier*Emissionsfaktor
frischfaser = 0.00484
teils = 0.00447

df['VerbrPap'] = 2*(df.Heft*16+df.College*80+df.Zeichen*40+df.Druck) # *2 wegen Halbjahren


for i in df.index:
    if (df.loc[i, 'Papierart'] == 'Recyclingpapier'):
        df.loc[i, 'CO2Pap'] = df.loc[i, 'VerbrPap'] * recycling
        df.loc[i, 'CO2PapRed'] = 0  # Keine Einsparungen, da bereits Recyclingpapier
    else:
        if (df.loc[i, 'Papierart'] == 'Frischfaserpapier'):
            df.loc[i, 'CO2Pap'] = df.loc[i, 'VerbrPap'] * frischfaser
        else:
            df.loc[i, 'CO2Pap'] = df.loc[i, 'VerbrPap'] * teils
        
        # Absolute Einsparungen
        einsparung = df.loc[i, 'CO2Pap'] - (df.loc[i, 'VerbrPap'] * recycling)
        df.loc[i, 'CO2PapRed'] = einsparung


df


,Ich_bin,Wegstrecke,Tage_Fuß,Tage_Rad,Tage_OPNV,Tage_Auto,Autoart,Heft,College,Zeichen,Druck,Papierart,FID,VerbrPap,CO2Pap,CO2PapRed
0,Lehrkraft,6.2,0.0,5.0,0.0,0.0,<NA>,0,0,4,2500,Recyclingpapier,1,5320,21.81200,0.00000
1,Lehrkraft,5.0,0.0,5.0,0.0,0.0,<NA>,0,10,0,2500,Recyclingpapier,2,6600,27.06000,0.00000
2,Schülerin oder Schüler,3.0,0.0,0.0,5.0,0.0,<NA>,0,0,0,20,Recyclingpapier,3,40,0.16400,0.00000
3,Schülerin oder Schüler,1.7,0.0,5.0,0.0,0.0,<NA>,6,2,1,0,"teils, teils",4,592,2.64624,0.21904
4,Schülerin oder Schüler,2.6,2.5,0.0,2.5,0.0,Diesel,4,3,1,24,"teils, teils",5,736,3.28992,0.27232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Schülerin oder Schüler,3.0,0.0,4.0,1.0,0.0,E-Auto,2,1,2,0,Recyclingpapier,174,384,1.57440,0.00000
174,Schülerin oder Schüler,1.3,2.0,3.0,0.0,0.0,<NA>,0,4,1,0,"teils, teils",175,720,3.21840,0.26640
175,Schülerin oder Schüler,0.5,0.0,5.0,0.0,0.0,<NA>,3,3,0,20,Recyclingpapier,176,616,2.52560,0.00000
176,Schülerin oder Schüler,5.0,0.0,0.0,5.0,0.0,<NA>,1,0,0,0,"teils, teils",177,32,0.14304,0.01184


### Berechnung Emissionen durch Mobilität

In [6]:
for i in df.index:
    autoart = df.loc[i, 'Autoart']
    if pd.isna(autoart):
        continue  # Überspringt die Iteration, wenn kein Auto vorhanden ist (N/A oder NaN)
    elif autoart == 'Benziner':
        df.loc[i, 'Autoart'] = 'Auto (Benzin)'
    elif autoart == 'Diesel':
        df.loc[i, 'Autoart'] = 'Auto (Diesel)'
    elif autoart == 'Auto _Mittel':
        df.loc[i, 'Autoart'] = 'Auto (Mittel)'


df

,Ich_bin,Wegstrecke,Tage_Fuß,Tage_Rad,Tage_OPNV,Tage_Auto,Autoart,Heft,College,Zeichen,Druck,Papierart,FID,VerbrPap,CO2Pap,CO2PapRed
0,Lehrkraft,6.2,0.0,5.0,0.0,0.0,<NA>,0,0,4,2500,Recyclingpapier,1,5320,21.81200,0.00000
1,Lehrkraft,5.0,0.0,5.0,0.0,0.0,<NA>,0,10,0,2500,Recyclingpapier,2,6600,27.06000,0.00000
2,Schülerin oder Schüler,3.0,0.0,0.0,5.0,0.0,<NA>,0,0,0,20,Recyclingpapier,3,40,0.16400,0.00000
3,Schülerin oder Schüler,1.7,0.0,5.0,0.0,0.0,<NA>,6,2,1,0,"teils, teils",4,592,2.64624,0.21904
4,Schülerin oder Schüler,2.6,2.5,0.0,2.5,0.0,Auto (Diesel),4,3,1,24,"teils, teils",5,736,3.28992,0.27232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Schülerin oder Schüler,3.0,0.0,4.0,1.0,0.0,E-Auto,2,1,2,0,Recyclingpapier,174,384,1.57440,0.00000
174,Schülerin oder Schüler,1.3,2.0,3.0,0.0,0.0,<NA>,0,4,1,0,"teils, teils",175,720,3.21840,0.26640
175,Schülerin oder Schüler,0.5,0.0,5.0,0.0,0.0,<NA>,3,3,0,20,Recyclingpapier,176,616,2.52560,0.00000
176,Schülerin oder Schüler,5.0,0.0,0.0,5.0,0.0,<NA>,1,0,0,0,"teils, teils",177,32,0.14304,0.01184


In [7]:
def calculate_co2(row):
    weg = row['Wegstrecke']
    tage_auto = row['Tage_Auto']
    tage_opnv = row['Tage_OPNV']
    tage_rad = row['Tage_Rad']
    tage_fuss = row['Tage_Fuß']
    auto_typ = row['Autoart']
    
    # CO2 Emissionen pro km (in kg CO2)
    emissions_opnv = 0.064  # ÖPNV Hamburg Mittel
    emissions_rad = 0.008   # Angenommen, durchschnittlich 0.008 kgCO2e/Pkm für Fahrräder
    emissions_fuss = 0      # Zu Fuß gehen verursacht keine CO2-Emissionen

    # Bestimmung des CO2-Emissionsfaktors basierend auf dem Autotyp
    emissionsfaktoren = {
        'Hybrid': 0.132,
        'Auto (Benzin)': 0.21,
        'Auto (Diesel)': 0.205,
        'E-Auto': 0.099
    }

    # Gewichteter Durchschnitt der Emissionen, falls Autotyp nicht bekannt
    gewichteter_durchschnitt = (0.21*0.623)+(0.205*0.326)+(0.132*0.025)+(0.099*0.013) + (0.1615*0.013)

    # CO2-Emissionen für das Auto berechnen
    co2_auto = tage_auto * weg * 2 * emissionsfaktoren.get(auto_typ, gewichteter_durchschnitt) * 38

    # CO2-Emissionen für andere Transportmittel berechnen
    co2_opnv = tage_opnv * weg * 2 * emissions_opnv * 38
    co2_rad = tage_rad * weg * 2 * emissions_rad * 38
    co2_fuss = tage_fuss * weg * 2 * emissions_fuss * 38

    # Summe der aktuellen Gesamtemissionen
    co2_aktuell = co2_auto + co2_opnv + co2_rad + co2_fuss

    # Potentielle Einsparungen berechnen
    if weg <= 5:
        # Alle motorisierten Tage durch Fahrrad ersetzen (niedrige Emissionen)
        co2_neu = co2_fuss+ co2_rad + (tage_opnv + tage_auto) * weg * 2 * emissions_rad * 38
    else:
        # Nur Autofahrten durch ÖPNV ersetzen
        
        co2_neu = co2_fuss + co2_rad + co2_opnv + (tage_auto * emissions_opnv) * weg * 2 *38

    # Einsparungen berechnen
    co2_einsparung = co2_aktuell - co2_neu

    # Rückgabe der aktuellen Gesamtemissionen und der Einsparungen
    return co2_aktuell, co2_einsparung

# Anwenden der Funktion und Zuweisung der Ergebnisse zu neuen Spalten
df[['CO2Mob', 'CO2Red']] = df.apply(calculate_co2, axis=1, result_type='expand')
df

,Ich_bin,Wegstrecke,Tage_Fuß,Tage_Rad,Tage_OPNV,Tage_Auto,Autoart,Heft,College,Zeichen,Druck,Papierart,FID,VerbrPap,CO2Pap,CO2PapRed,CO2Mob,CO2Red
0,Lehrkraft,6.2,0.0,5.0,0.0,0.0,<NA>,0,0,4,2500,Recyclingpapier,1,5320,21.81200,0.00000,18.8480,0.000
1,Lehrkraft,5.0,0.0,5.0,0.0,0.0,<NA>,0,10,0,2500,Recyclingpapier,2,6600,27.06000,0.00000,15.2000,0.000
2,Schülerin oder Schüler,3.0,0.0,0.0,5.0,0.0,<NA>,0,0,0,20,Recyclingpapier,3,40,0.16400,0.00000,72.9600,63.840
3,Schülerin oder Schüler,1.7,0.0,5.0,0.0,0.0,<NA>,6,2,1,0,"teils, teils",4,592,2.64624,0.21904,5.1680,0.000
4,Schülerin oder Schüler,2.6,2.5,0.0,2.5,0.0,Auto (Diesel),4,3,1,24,"teils, teils",5,736,3.28992,0.27232,31.6160,27.664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,Schülerin oder Schüler,3.0,0.0,4.0,1.0,0.0,E-Auto,2,1,2,0,Recyclingpapier,174,384,1.57440,0.00000,21.8880,12.768
174,Schülerin oder Schüler,1.3,2.0,3.0,0.0,0.0,<NA>,0,4,1,0,"teils, teils",175,720,3.21840,0.26640,2.3712,0.000
175,Schülerin oder Schüler,0.5,0.0,5.0,0.0,0.0,<NA>,3,3,0,20,Recyclingpapier,176,616,2.52560,0.00000,1.5200,0.000
176,Schülerin oder Schüler,5.0,0.0,0.0,5.0,0.0,<NA>,1,0,0,0,"teils, teils",177,32,0.14304,0.01184,121.6000,106.400


### Daten hochladen

Feature-Layer enthalten immer geografische Informationen. Unsere Datentabelle hat jedoch keine Geometrie. Um die Daten dennoch als Feature-Layer abspeichern zu können, fügen wir Dummy-Geometrien hinzu. Dies macht den Layer technisch gesehen "räumlich", sodass er in ArcGIS Online veröffentlicht werden kann, ohne dass er tatsächlich geografisch relevant ist. 

Mit `spatial.to_featurelayer()` kann ein Feature-Layer erstellt werden. Diese Methode muss nur beim ersten Ausführen des Skripts verwendet werden. Sobald der Feature-Layer erstellt ist, kann er mit `searchoverwrite()` überschrieben werden.

Im oberen Reiter der ArcGIS Online-Notebooks gibt es den Button `Tasks`, mit dem sich Skriptausführungen automatisieren lassen. Wenn Berechnungen zusammen mit den Umfrageergebnissen nahezu in Echtzeit visualisiert werden sollen, kann dies darüber eingestellt werden.

In [8]:
df['longitude'] = 0.0
df['latitude'] = 0.0

# Schritt 1: In einen Spatial DataFrame umwandeln
sdf = pd.DataFrame.spatial.from_xy(df, 'longitude', 'latitude')


# Schritt 2: Feature Layer in ArcGIS Online erstellen ACHTUNG: Diese Zeile muss nur beim erstmaligen Run ausgeführt werden. Danach auskommentieren und die Funktion searchoverwrite verwenden
feature_layer = sdf.spatial.to_featurelayer(title="Testdaten_Ergebnisse", gis=gis)


In [9]:
# ab dem zweiten Run
#searchoverwrite('Testdaten_Ergebnisse',sdf)